In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# 한글폰트
import koreanize_matplotlib
plt.rc('font', family = 'Malgun Gothic')
plt.rcParams['axes.unicode_minus'] = False

# warnings제거
import warnings

In [2]:
# 서비스 계정 키 경로 설정
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = r"C:\Users\jey34\OneDrive\바탕 화면\mission18\sprintda05-eunyu-a9946f7dea9c.json"

# GCS의 Parquet 파일 경로
gcs_path = 'gs://final_project_enuyu/data/final_project/votes/event_receipts.parquet'

# 데이터 불러오기
event_receipts_df = pd.read_parquet(gcs_path, engine='pyarrow')
event_receipts_df

,id,created_at,event_id,user_id,plus_point
0,2,2023-06-22 09:25:16,1,1193618,500
1,3,2023-06-22 09:38:53,1,928351,500
2,4,2023-06-22 10:32:15,1,904872,500
3,5,2023-06-22 13:03:06,1,974697,500
4,6,2023-06-22 13:40:38,1,1168260,500
...,...,...,...,...,...
304,317,2023-11-14 11:17:29,3,918053,500
305,318,2023-11-15 05:09:02,3,932419,500
306,319,2023-11-17 06:17:25,3,1188104,500
307,320,2023-11-17 06:17:59,3,1296978,500
